## Ingest drivers.json file

1. Read the JSON file using the spark dataframe reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import concat, lit, col

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
file_date= dbutils.widgets.get("p_file_date")

In [0]:
name_schema = StructType(
    fields=[
        StructField("forename", StringType(), True),
        StructField("surname", StringType(), True),
    ]
)
driver_schema = StructType(
    fields=[
        StructField("driverId", IntegerType(), False),
        StructField("driverRef", StringType(), True),
        StructField("number", IntegerType(), True),
        StructField("code", StringType(), True),
        StructField("name", name_schema, True),
        StructField("dob", DateType(), True),
        StructField("nationality", StringType(), True),
        StructField("url", StringType(), True),
    ]
)

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
driver_df = spark.read \
  .schema(driver_schema) \
  .json(f'{raw_folder_path}/{file_date}/drivers.json')
#display(driver_df)

2. Rename columns, add ingestion date and clean name column

In [0]:
driver_renamed_df = driver_df.withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("driverRef", "driver_ref") \
    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
    .drop('url') \
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(file_date))
driver_final_df = add_ingestion_date(driver_renamed_df)
#display(driver_renamed_df)

3. Write data to datalake as parquet

In [0]:
#driver_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/drivers")
# database
driver_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.drivers")

In [0]:
# %fs
# ls mnt/formula1jf/processed/drivers

In [0]:
dbutils.notebook.exit("Success")